# Sort on institutions

In [111]:
# libraries
import pandas as pd
import re
from itertools import chain
pd.options.mode.chained_assignment = None

# data load
df = pd.read_csv('kkorg_enheder_offentlig.csv', delimiter=';')

In [112]:
# Apply lambda function to each row to extract longitude and latitude
def extract_coordinates(row):
    # Define a regular expression pattern to extract coordinates
    pattern = r'[\d.]+'

    matches = re.findall(pattern, row)
    if len(matches) >= 2:
        return pd.Series((float(matches[0]), float(matches[1])))
    else:
        return pd.Series((None, None))


# Save the extracted coordinates in two new columns
df['wkb_geometry'] = df['wkb_geometry'].astype(str)
df[['longitude', 'latitude']] = df['wkb_geometry'].apply(extract_coordinates)

In [113]:
# find the rows where the column 'enhedstype' is 'Vuggestuer', 'Dagplejemødre', 'Børnehaver', 'Folkeskoler o.lign.', 'Aldersintegrerede institutioner'
filter_df = df[df['enhedstype'].isin(['Vuggestuer', 'Dagplejemødre', 'Børnehaver', 'Folkeskoler o.lign.', 'Aldersintegrerede institutioner'])]
filter_df['type_split'] = filter_df['type'].str.split(', ')
filter_df = filter_df.explode('type_split')
filter_df = filter_df[filter_df['type_split'].str.contains('Vuggestue|Dagpleje|Flerbørnsdagpleje|Børnehave|Distriktsskole')]
filter_df = filter_df.reset_index(drop=True)

# remove duplicates and rows with missing values
filter_df = filter_df.drop_duplicates(subset=['id', 'enhedsnavn', 'type_split'])
filter_df = filter_df[filter_df['longitude'].notna() & filter_df['latitude'].notna()]

In [176]:
filter_df.columns

filter_df[['adresse']]

,adresse
0,"Læssøesgade 22, 2200 København N"
1,"Prinsessegade 45, 1422 København K"
2,"Frederiksholms Kanal 28D, 1220 København K"
3,"Blommestien 8, 2500 Valby"
4,"Estlandsgade 4, 1724 København V"
...,...
885,"Fjenneslevvej 1, 2700 Brønshøj"
886,"Sjællandsgade 10, 2200 København N"
887,"Vallekildevej 88, 2700 Brønshøj"
888,"Carl Jacobsens Vej 23B, 2500 Valby"


In [184]:
import folium
import shapely.geometry as sg


# Create the Folium map for Copenhagen
latitude = 55.676  # Example latitude
longitude = 12.568  # Example longitude
zoom_start = 11.5  # Initial zoom level
m = folium.Map(location=[latitude, longitude], zoom_start=zoom_start)

# plot schools as markers with popups according to the type
for index, row in filter_df.iterrows():
    name = row['enhedsnavn']
    coordinates = (row['latitude'], row['longitude'])
    address = row['adresse']
    link = row['www']
    if pd.isna(link):
        popup_content = '<div style="width: 200px;"><b>' + name + '</b><br>' + row['type_split'] + '<br>' + address + '</div>'
    else:
        popup_content = '<div style="width: 200px;"><b>' + name + '</b><br>' + row['type_split'] + '<br>' + address + '<br><a href="' + link + '" target="_blank">Link</a></div>'

    if 'Vuggestue' in row['type_split'] or 'Dagpleje' in row['type_split'] or 'Flerbørnsdagpleje' in row['type_split']:
        folium.Marker(location=coordinates, popup=popup_content, icon=folium.Icon(color='lightblue', icon='fa-baby', prefix='fa')).add_to(m)
    elif 'Børnehave' in row['type_split']:
        folium.Marker(location=coordinates, popup=popup_content, icon=folium.Icon(color='blue', icon='fa-hippo', prefix='fa')).add_to(m)
    elif 'Distriktsskole' in row['type_split']:
        folium.Marker(location=coordinates, popup=popup_content, icon=folium.Icon(color='darkblue', icon='fa-school', prefix='fa')).add_to(m)

In [185]:
# Define JavaScript to toggle markers based on checkbox state
js = """
<script>
function updateMarkers() {
    var vuggestueChecked = document.getElementById('vuggestueCheckbox').checked;
    var børnehaveChecked = document.getElementById('børnehaveCheckbox').checked;
    var distriktsskoleChecked = document.getElementById('distriktsskoleCheckbox').checked;
    var includeTypes = [];
    if (vuggestueChecked) {
        includeTypes.push('Vuggestue');
    }
    if (børnehaveChecked) {
        includeTypes.push('Børnehave');
    }
    if (distriktsskoleChecked) {
        includeTypes.push('Distriktsskole');
    }
    document.getElementById('map').innerHTML = '';
    filterMarkers(includeTypes);
}
</script>
"""

# Add checkboxes to the map
checkboxes_html = """
<div id='map-checkboxes'>
    <input type='checkbox' id='vuggestueCheckbox' checked onclick='updateMarkers()'> Vuggestue<br>
    <input type='checkbox' id='børnehaveCheckbox' checked onclick='updateMarkers()'> Børnehave<br>
    <input type='checkbox' id='distriktsskoleCheckbox' checked onclick='updateMarkers()'> Distriktsskole<br>
</div>
"""
m.get_root().html.add_child(folium.Element(checkboxes_html))

# Add the JavaScript code to the map
m.get_root().html.add_child(folium.Element(js))

In [186]:
# Display the map
m